# Book Recommendations.

In [48]:
import sqlite3
from contextlib import contextmanager
from typing import List, Tuple, Dict


class BooksBD:

    def __init__(self) -> None:
        self.con = sqlite3.connect("books.db")
        self.cur = self.con.cursor()
        self.cols = set([
            "title",
            "description",
            "url",
            "image_url",
        ])
        self.cur.execute('''
        CREATE TABLE IF NOT EXISTS books (
              id integer not null primary key asc
            , title text
            , description text
            , url text
            , image_url text
        )
        ;
        ''')
        self.con.commit()
        
        
    def __enter__(self) -> BooksBD:
        return self

    def add_books(self,
                  cols:List[str],
                  books:List[Tuple[str, str]]) -> None:
        vals = ",".join(["?" for _ in cols])
        cols = ",".join([c for c in cols if c in self.cols])
        self.cur.executemany(f'''
        INSERT INTO books ({cols}) values ({vals})
        ''', books)
        self.con.commit()
        
    def update_by_id(self, _id:int, values:Dict[str, str]) -> None:
        k_v = ",".join([f"{k}=\"{v}\"" \
                        for k,v in values.items() if k in self.cols])
        self.cur.execute(f'''
        UPDATE books SET {k_v} WHERE id = ?
        ''', (_id,))
        self.con.commit()
                
    def get_by_title(self, title:str) -> List[Tuple[int, str, str]]:
        return [book for book in self.cur.execute('''
        SELECT * FROM books where title like ?
        ''', ("%"+title+"%",))]
    
    def get_by_id(self, _id:int) -> Tuple[int, str, str]:
        return [book for book in self.cur.execute('''
        SELECT * FROM books where id = ?
        ''', (_id,))]
    
    def __exit__(self, exc_type, exc_val, exc_tb) -> None:
        self.con.close()

In [49]:
with BooksBD() as db:
    db.add_books(["title", "description"], [
        ("Title A", "Description title A."),
        ("Title B", "Description title B."),
    ])

In [50]:
with BooksBD() as db:
    db.update_by_id(1, {"url":"url updated!"})

In [51]:
with BooksBD() as db:
    print(db.get_by_title("Title"))

[(1, 'Title A', 'Description title A.', 'url updated!', None), (2, 'Title B', 'Description title B.', None, None)]


In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-mpnet-base-v2") # English.
# model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2") # Multilingual.

In [23]:
Books = model.encode([
    "Treasure Islands: Tax Havens and the Men Who Stole the World. This is the ugliest chapter in global economic affairs since slavery -- and secretive offshore tax havens are at the heart of the trouble. Dirty money, tax havens and the offshore system describe the ugliest and most secretive chapter in the history of global economic affairs. The World Bank has reported that the flow of dirty money across borders, out of developing countries and into rich ones, is up to ten times the amount of foreign aid that flows the other way. Most people regard tax havens as being relevant only to celebrities, crooks and spivs, and mistakenly believe that the main offshore problems are money laundering and terrorist financing. These are only small parts of the whole picture. The offshore system has been (discreetly) responsible for the greatest ever shift of wealth from poor to rich. It also undermines our democracies by offering the wealthiest members of society escape routes from tax, financial regulation and other normal democratic controls. The anti-globalisation movement tapped into an uneasiness, felt by millions, that something was rotten in the world economy. Treasure Islands brilliantly articulates the problem in a completely new way, and exposes the deep corruption that impacts on our daily lives.",
    "The book of longings. Raised in a wealthy family in Sepphoris with ties to the ruler of Galilee, Ana is rebellious and ambitious, a relentless seeker with a brilliant, curious mind and a daring spirit. She yearns for a pursuit worthy of her life, but finds no outlet for her considerable talents. Defying the expectations placed on women, she engages in furtive scholarly pursuits and writes secret narratives about neglected and silenced women. When she meets the eighteen-year-old Jesus, each is drawn to and enriched by the other’s spiritual and philosophical ideas. He becomes a floodgate for her intellect, but also the awakener of her heart. Their marriage unfolds with love and conflict, humor and pathos in Nazareth, where Ana makes a home with Jesus, his brothers, James and Simon, and their mother, Mary. Here, Ana’s pent-up longings intensify amid the turbulent resistance to the Roman occupation of Israel, partially led by her charismatic adopted brother, Judas. She is sustained by her indomitable aunt Yaltha, who is searching for her long-lost daughter, as well as by other women, including her friend Tabitha, who is sold into slavery after she was raped, and Phasaelis, the shrewd wife of Herod Antipas. Ana’s impetuous streak occasionally invites danger. When one such foray forces her to flee Nazareth for her safety shortly before Jesus’s public ministry begins, she makes her way with Yaltha to Alexandria, where she eventually finds refuge and purpose in unexpected surroundings. Grounded in meticulous historical research and written with a reverential approach to Jesus’s life that focuses on his humanity, The Book of Longings is an inspiring account of one woman’s bold struggle to realize the passion and potential inside her, while living in a time, place, and culture devised to silence her.",
    "A Feast of Sorrows. A Feast of Sorrows—Angela Slatter’s first U.S. collection—features twelve of the World Fantasy and British Fantasy Award-winning Australian author’s finest, darkest fairy tales, and adds two new novellas to her marvelous cauldron of fiction. Stories peopled by women and girls—fearless, frightened, brave, bold, frail, and fantastical—who take the paths less traveled by, accept (and offer) poisoned apples, and embrace transformation in all its forms. Reminiscent of Angela Carter at her best, Slatter’s work is both timeless and fresh: fascinating new reflections from the enchanted mirrors of fairy tales and folklore.",
    "The Big Short: Inside the Doomsday Machine. The #1 New York Times bestseller: \"It is the work of our greatest financial journalist, at the top of his game. And it's essential reading.\"—Graydon Carter, Vanity Fair The real story of the crash began in bizarre feeder markets where the sun doesn't shine and the SEC doesn't dare, or bother, to tread: the bond and real estate derivative markets where geeks invent impenetrable securities to profit from the misery of lower- and middle-class Americans who can't pay their debts. The smart people who understood what was or might be happening were paralyzed by hope and fear; in any case, they weren't talking. Michael Lewis creates a fresh, character-driven narrative brimming with indignation and dark humor, a fitting sequel to his #1 bestseller Liar's Poker. Out of a handful of unlikely-really unlikely-heroes, Lewis fashions a story as compelling and unusual as any of his earlier bestsellers, proving yet again that he is the finest and funniest chronicler of our time.",
    "Too Big to Fail: The Inside Story of How Wall Street and Washington Fought to Save the Financial System from Crisis — and Themselves. Andrew Ross Sorkin delivers the first true behind-the-scenes, moment-by-moment account of how the greatest financial crisis since the Great Depression developed into a global tsunami. From inside the corner office at Lehman Brothers to secret meetings in South Korea, and the corridors of Washington, Too Big to Fail is the definitive story of the most powerful men and women in finance and politics grappling with success and failure, ego and greed, and, ultimately, the fate of the world’s economy. “We’ve got to get some foam down on the runway!” a sleepless Timothy Geithner, the then-president of the Federal Reserve of New York, would tell Henry M. Paulson, the Treasury secretary, about the catastrophic crash the world’s financial system would experience. Through unprecedented access to the players involved, Too Big to Fail re-creates all the drama and turmoil, revealing neverdisclosed details and elucidating how decisions made on Wall Street over the past decade sowed the seeds of the debacle. This true story is not just a look at banks that were “too big to fail,” it is a real-life thriller with a cast of bold-faced names who themselves thought they were too big to fail.",
])

Books.shape

(5, 768)

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(Books[:2], Books[2:])

array([[0.18975008, 0.38999668, 0.39831614],
       [0.47643942, 0.14825481, 0.13352697]], dtype=float32)